In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from autograd import elementwise_grad as ele_grad
from tqdm.notebook import tqdm
from model import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
USE_GPU = True

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)
print(torch.__version__)

using device: cpu
1.11.0+cu102


In [305]:
np.random.seed(345)
torch.manual_seed(345)

epochs = 1000
n = 12

Q = np.random.randn(n, n)
S = Q.T @ Q

x0 = np.random.randn(n)

# lr_cos = optim.lr_scheduler.CosineAnnealingLR

EigenSolver = FullyConnectedN(n, [120, 120, 120], nonlinearity="relu")
optimizer = optim.Adam(EigenSolver.parameters(), lr=0.0002)
# scheduler = lr_cos(optimizer, epochs)


In [306]:
eigen = train(EigenSolver, optimizer, S, x0, 1e-2, epochs, device, dtype)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [307]:
data = compare_eig(S, eigen)
print(data["val_err"])
print(data["vec_err"])
print(data["eval_np"], data["eval_NN"])
# print(data["evec_np"])
# print(data["evec_NN"])

0.0207013883800522
1.1519955797679629
1.5384399222089058 1.559141310588958


In [185]:
np.random.seed(226)
torch.manual_seed(226)

epochs = 50
n = 100
Toeplitz = np.tri(n, k=1) - np.tri(n, k=-2) + np.eye(n)
# Q = np.random.randn(n, n)
# S = Q.T @ Q
S = Toeplitz

x0 = np.random.randn(n)

# lr_scheduler = optim.lr_scheduler.CosineAnnealingLR

EigenSolver = FullyConnectedN(n, [300, 300, 120, 120, 120], nonlinearity="relu")
optimizer = optim.AdamW(EigenSolver.parameters(), lr=0.03)
# scheduler = lr_scheduler(optimizer, epochs, 0)

In [188]:
eigen = train(EigenSolver, optimizer, S, x0, 1e4, epochs, device, dtype)


  0%|          | 0/50 [00:00<?, ?it/s]

In [189]:
data = compare_eig(S, eigen)
print(data["val_err"])
print(data["vec_err"])
print(data["eval_np"], data["eval_NN"])

0.026328055479432955
0.28214202598037563
2.0932807807748324 2.0669527252953994
